In [1]:
import pandas as pd
import string
import nltk
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, precision_recall_curve, accuracy_score
from sklearn.metrics import roc_auc_score
# !pip install imblearn --quiet
# !pip install llm2vec --quiet
# !pip install flash-attn --no-build-isolation
# import torch
# from llm2vec import LLM2Vec


import warnings

warnings.filterwarnings('ignore')

In [ ]:
from huggingface_hub import login
import pickle
from dotenv import load_dotenv
import os

load_dotenv()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
test_data = pd.read_csv('/content/drive/MyDrive/Agent_LLM_paper/drugsComTest_raw.tsv',sep = '\t')
train_data = pd.read_csv('/content/drive/MyDrive/Agent_LLM_paper/drugsComTrain_raw.tsv',sep = '\t')

In [6]:
test_data['label'] = test_data['rating'].apply(lambda x: 0 if x <= 5 else 1)
train_data['label'] = train_data['rating'].apply(lambda x: 0 if x <= 5 else 1)

In [7]:
test_data['clean_review'] = test_data['review'].str.lower()
train_data['clean_review'] = train_data['review'].str.lower()

In [ ]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in STOPWORDS])
def remove_spl_chars(text):
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    text = re.sub('\s+', ' ', text)
    return text

train_data['clean_review'] = train_data['clean_review'].apply(lambda x: remove_stopwords(x))
train_data['clean_review'] = train_data['clean_review'].apply(lambda x: remove_spl_chars(x))

test_data['clean_review'] = test_data['clean_review'].apply(lambda x: remove_stopwords(x))
test_data['clean_review'] = test_data['clean_review'].apply(lambda x: remove_spl_chars(x))

In [8]:
y_train, y_test = train_data['label'], test_data['label']

In [ ]:
instruction = "Classify a given drug review text based as either positive or negative: "

In [ ]:
token = os.getenv("HUGGINGFACE_TOKEN")
os.environ["HUGGINGFACE_TOKEN"] = os.getenv("HUGGINGFACE_TOKEN")

In [ ]:
login(token=token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!huggingface-cli login --token "$HUGGINGFACE_TOKEN"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model = LLM2Vec.from_pretrained(
    "McGill-NLP/LLM2Vec-Mistral-7B-Instruct-v2-mntp",
    peft_model_name_or_path="McGill-NLP/LLM2Vec-Mistral-7B-Instruct-v2-mntp-supervised",
    device_map="cuda" if torch.cuda.is_available() else "cpu",
    torch_dtype=torch.bfloat16
)


def append_instruction(instruction, sentences):
    new_sentences = []
    for s in sentences:
        new_sentences.append([instruction, s, 0])
    return new_sentences


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/786 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

In [ ]:
print(f"Encoding {len(train_data)} training sentences...")
sentences_train = append_instruction(instruction, train_data['clean_review'])
x_train = np.asarray(model.encode(sentences_train, batch_size=32))

print(f"Encoding {len(test_data)} test sentences...")
sentences_test = append_instruction(instruction, test_data['clean_review'])
x_test = np.asarray(model.encode(sentences_test, batch_size=32))

Encoding 161297 training sentences...


Batches:   0%|          | 0/5041 [00:00<?, ?it/s]

Encoding 53766 test sentences...


Batches:   0%|          | 0/1681 [00:00<?, ?it/s]

In [ ]:
with open('/content/drive/MyDrive/Agent LLM paper/x_train_llm2vec.pkl', 'wb') as fichier:
    pickle.dump(x_train, fichier)

with open('/content/drive/MyDrive/Agent LLM paper/x_test_llm2vec.pkl', 'wb') as fichier:
    pickle.dump(x_test, fichier)